In [1]:
import numpy as np
import json
import os
from pathlib import Path
import shutil
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [7]:
# data_root = Path("../../Data/Words/charts1_fine_tuned/")
# dest_root = Path("../../TRAIN_DATA/Words_fine_tuned")

data_root = Path("../../Data/Letters_2")
dest_root = Path("../../TRAIN_DATA/Letters_2")

In [8]:
subfolders = list(data_root.glob("./charts*"))
if len(subfolders)==0 :
    subfolders = [data_root]

subfolders

[WindowsPath('../../Data/Letters_2')]

In [9]:
def create_dir(path:Path|str):
    if not os.path.exists(path):
        os.makedirs(path)
        
def xyxy2xywh(xyxy:np.ndarray, img_width, img_height):
    if xyxy.ndim==1:
        xyxy = xyxy[None]
    w = (xyxy[:, 2] - xyxy[:, 0]) / img_width
    h = (xyxy[:, 3] - xyxy[:, 1]) / img_height
    
    x = xyxy[:, 0]/img_width + w/2
    y = xyxy[:, 1]/img_height + h/2
    
    return np.stack((x, y, w, h)).T



def get_segmentation_from_box(box):
    x0,y0,w,h = box
    t = y0
    b = y0 + h
    l = x0
    r = x0 + w

    # [x0,y0, x1,y1, x2,y2, x3,y3]
    return [l,t, r,t, r,b, l,b]

def convert_coco_to_yolo(image_paths, annotation_path):
    with open(annotation_path, "r") as f:
        json_labels = json.load(f)
    data = {}
    
    for i, annot in enumerate(json_labels["annotations"]):
        img_id = annot["image_id"]

        json_image =  json_labels["images"][img_id-1] ## -1 is because the indexing starts from 1

        img_width = json_image["width"]
        img_height = json_image["height"]

        file_name = Path(json_image["file_name"])

        category = annot["category_id"] - 1
        if len(annot["segmentation"])==0:
            segments = np.array(get_segmentation_from_box(annot["bbox"]), dtype=float)
        else:
            segments = np.array(annot["segmentation"][0], dtype=float)
        bbox = np.array(annot["bbox"], dtype=float)
        
        # Rescaling
        segments[::2] /= img_width # Dividing x 
        segments[1::2] /= img_height # Dividing y 
        bbox = xyxy2xywh(bbox, img_width, img_height)[0] # Rescale and Convert to xywh
        segment_text = np.array2string(segments)[1:-1].replace('\n', '') + "\n"
        label_string = f"{category} {segment_text}"

        if img_id not in data.keys():
            data[img_id] = {}
            data[img_id]["segments"] = []

        data[img_id]["filename"] = file_name
        data[img_id]["segments"].append(label_string)        
    
    return data

def save_data(img_ids, data, dest_root, image_root, split="train"):  
    dst = Path(dest_root/split)
    
    for p in [dst, dst/"labels", dst/"images"]:
        create_dir(p)
    for img_id in img_ids:
        filename = data[img_id]["filename"]
        segments = data[img_id]["segments"]
        with open(dst/"labels"/(filename.stem+".txt"), "w") as f:
            f.writelines(segments)

        shutil.copy(image_root/(filename.stem + filename.suffix), dst/"images/")

def convert_save(subfolders, dest_root, train_data_amount=0.8):
    for subfolder in tqdm(subfolders):
        image_paths = list(subfolder.glob("images/*"))
        annotation_path = list(subfolder.glob("annotations/*.json"))[0]

        print(annotation_path)
        data = convert_coco_to_yolo(image_paths, annotation_path)
        image_ids = list(data.keys())
        np.random.shuffle(image_ids)
        split_index = int(len(image_ids)*train_data_amount)


        train_ids = image_ids[:split_index]
        val_ids = image_ids[split_index:]

        save_data(train_ids, data, dest_root, subfolder/"images", split="train")
        save_data(val_ids, data, dest_root, subfolder/"images", split="val")

In [10]:
convert_save(subfolders, dest_root, train_data_amount=0.95)

  0%|          | 0/1 [00:00<?, ?it/s]

..\..\Data\Letters_2\annotations\instances_default.json
